# Title Here

author1,author2,author3,author4

# Research Background

The foreign exchange (FX) market is one of the world's largest financial markets, with daily trading volumes that can reach trillions of dollars.

The carry trade is a popular trading strategy in the FX market, from where the investors invest in high-interest currencies by borrowing in low-interest currencies with the purpose of earning the interest rate differential. 

## Research Target

Based on the methodologies of quantitative time series analysis, we reserach the TRI series of AUD-USD currency pair, aim to fitting a significant model with historical data and getting solid predictions of future.

## Carry trade

In carry trade, we always hope to with "positive carry", i.e. "borrow" in low-interest currency and "lend" in high-interest currency.

![carry](../img/carry.png)

## TRI series

Our research is based on the daily TRI series of the AUD-USD currency pair from 1997 to 2022. TRI means the total return index of carry trade, which equals to the sum of spot change and cumulative carry. 

$$
TRI_{t} = SPOT_{t} + \sum_{i=1}^{t}CARRY_{i}
$$

<img src="../img/tri.png" style="zoom:.8">

### Preprocessing

Let time start at $t = 0$. Denote the TRI's time series by $TRI(t)$, then compute its log return $r(t)$ with

$$
r(t) = \ln \left[\frac{TRI(t)}{TRI(t-1)}\right].
$$

The log return of $TRI$ series are shown in:

![](../img/log_return.png)

In order to check the autocorrelation of the log return series of TRI, we perform the Ljung-Box test and plot the ACF and PACF of the log return series of TRI:

![](../img/acfs.png)

| lag | lb_stat    | lb_pvalue    |
|:---:|:----------:|:------------:|
| 1   | 30.394860  | 3.524609e-08 |
| 2   | 34.212607  | 3.722433e-08 |
| 3   | 143.265224 | 7.470724e-31 |
| 4   | 210.249295 | 2.348345e-44 |
| 5   | 226.225141 | 6.889873e-47 |

### Model Selection

Although, we can guess the hyperparameters of the ARMA model from the ACF and PACF, we still want to find the best hyperparameters by carry out a grid search. We select the hyperparameters $(p,q) \in ([1,10]\bigcap \mathbb{R})\times ([1,10]\bigcap \mathbb{R})$. 

<center><img src="../img/aic.png" style="zoom:.4"></center>

We choose the model with the smallest AIC as the best model. The best model is ARMA(5,5).

### Prediction on train set 

The plot of model prediction on train set is shown below

![](../img/arima_train_pred.png)

### Model Checking

In order to check whether the model satisfies the white noise assumption, we carry out the "arima_diagnostics" and the Ljung-Box test for residuals, the plot is shown below:

![](../img/arima_diagnostics.png)

### Forecasting

In practice, the data update every day. So we fixs the paramaters fitted by train set and use the test data before every next day to predict the log return of the next day. The plot of the rolling-forecasting is shown below:

![](../img/arima_forecast.png)

## Check ARCH effect

To further study, we also check the ARCH effect for the residuals. We carry out the Ljung-Box test for the squared residuals $\hat{a}_t^2$. The test results are shown below:

| lag | lb_stat     | lb_pvalue     |
| --- | ----------- | ------------- |
| 1   | 655.473127  | 1.441523e-144 |
| 2   | 1130.314543 | 3.591828e-246 |
| 3   | 1577.095845 | 0.000000e+00  |
| 4   | 3110.276390 | 0.000000e+00  |
| 5   | 3753.389311 | 0.000000e+00  |

From the table, we can see that the p-value of the Ljung-Box test is very small, which means that the squared residuals $\hat{a}_t^2$ are serial correlated. The ARCH effect exists in the residuals. Then we should consider the GARCH model to study the conditional violatility in the log return series.

## GARCH Model

We fit the GARCH(1,1) model with residuals $\hat{a}_t$ we get in the arima model.

$$
\begin{aligned}
a_t&=\sigma_t \epsilon_t\\
\sigma_t^2&=\omega+\alpha a_{t-1}^2+\beta\sigma_{t-1}^2
\end{aligned}
$$

where $\epsilon_t\sim N(0,1)$.

The fitted standardized residuals $\hat{\epsilon}_t$ and the fitted violatility $\hat{\sigma}_t$ are shown below:

![](../img/garch_result.png)

### Prediction on train set
The plot of prediction of violatility $\hat{\sigma}_t$ and ground truth of log return $r_t$ is shown below:

![](../img/garch_train_pred.png)

### Model Checking

In order to check whether the standardized residuals $\hat{\epsilon}_t$ satisfies the white noise assumption, we carry out the "garch_diagnostics". We plot the acf, pacf, histplot qqplot of the standardized residuals. The plot is shown below:

![](../img/garch_diagnostics.png)